In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [116]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response=requests.get(wikiurl)
print(response.status_code)

200


In [117]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
main_table=soup.find('table',{'id':"constituents"})

In [118]:
df=pd.read_html(str(main_table))
# convert list to dataframe
df=pd.DataFrame(df[0])
print(df.head())

  Symbol     Security SEC filings  GICS Sector         GICS Sub-Industry  \
0    MMM           3M     reports  Industrials  Industrial Conglomerates   
1    AOS  A. O. Smith     reports  Industrials         Building Products   
2    ABT       Abbott     reports  Health Care     Health Care Equipment   
3   ABBV       AbbVie     reports  Health Care           Pharmaceuticals   
4   ABMD      Abiomed     reports  Health Care     Health Care Equipment   

     Headquarters Location Date first added      CIK      Founded  
0    Saint Paul, Minnesota       1976-08-09    66740         1902  
1     Milwaukee, Wisconsin       2017-07-26    91142         1916  
2  North Chicago, Illinois       1964-03-31     1800         1888  
3  North Chicago, Illinois       2012-12-31  1551152  2013 (1888)  
4   Danvers, Massachusetts       2018-05-31   815094         1981  


#### Getting Company Wiki Links

Get all the tags within the table that have a wiki link.

Ex. `<a href="/wiki/Accenture" title="Accenture">Accenture</a>`

In [119]:
WIKI = "\/wiki\/([\w]+)"
wiki_link_tags = main_table.findAll('a', attrs={'href': re.compile(WIKI)})
wiki_link_tags = wiki_link_tags[4:] # Jump to the first company
wiki_link_tags[:5]

[<a href="/wiki/3M" title="3M">3M</a>,
 <a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a>,
 <a href="/wiki/A._O._Smith" title="A. O. Smith">A. O. Smith</a>,
 <a class="mw-redirect" href="/wiki/Milwaukee,_Wisconsin" title="Milwaukee, Wisconsin">Milwaukee, Wisconsin</a>,
 <a href="/wiki/Abbott_Laboratories" title="Abbott Laboratories">Abbott</a>]

Now we only want to get the tags of actual companies.

In [120]:
company_names_from_df = df.Security.values

company_tags = []
company_names_from_tags = []
for tag in wiki_link_tags: 
    name = tag.contents[0]

    if name in company_names_from_df:
        company_tags.append(tag)
        company_names_from_tags.append(name)
        
print(company_tags[:5])
print(company_names_from_tags[:5])

[<a href="/wiki/3M" title="3M">3M</a>, <a href="/wiki/A._O._Smith" title="A. O. Smith">A. O. Smith</a>, <a href="/wiki/Abbott_Laboratories" title="Abbott Laboratories">Abbott</a>, <a href="/wiki/AbbVie" title="AbbVie">AbbVie</a>, <a href="/wiki/Abiomed" title="Abiomed">Abiomed</a>]
['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Abiomed']


In [121]:
# Companies that are in the dataframe but not in the names we got
set(company_names_from_df) - set(company_names_from_tags)

{'Alphabet Inc. (Class A)',
 'Alphabet Inc. (Class C)',
 'Fox Corporation (Class A)',
 'Fox Corporation (Class B)',
 'News Corp (Class A)',
 'News Corp (Class B)',
 'Pool Corporation'}

Now we get the wikipedia links of all the companies in the list of tags. We can use the `href` attribute. We make this a mapping of company name to link.

In [122]:
WIKI_LINK_BEGINNING = "https://en.wikipedia.org"  # All wikipedia links start like this

wiki_links_map = {}
for tag in company_tags:
    name = tag.contents[0]
    link = WIKI_LINK_BEGINNING + tag['href']
    wiki_links_map[name] =  link

Add this links to the dataframe.

In [123]:
def get_wiki_link(row):
    try:
        return wiki_links_map[row.Security]
    except:
        return None
df["Wiki Link"] = df.apply(lambda row: get_wiki_link(row), axis=1)
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,https://en.wikipedia.org/wiki/3M
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,https://en.wikipedia.org/wiki/Abiomed


#### Getting Official Website from Wiki Page: Method 1

1. Get the soup for the company wiki page
2. Search for a `span` tag with a class "official-website"
3. Find `a` tag within that and get the href associated with it

Example on 3M wiki page

In [124]:
threeM = "https://en.wikipedia.org/wiki/3M"
threeM_response=requests.get(threeM)
print(threeM_response.status_code)
response_soup = BeautifulSoup(threeM_response.text, 'html.parser')
threeM_tag = response_soup.find('span', {'class':"official-website"})

threeM_tag.find('a')['href']

200


'https://www.3m.com/'

Make a function that does this.

In [125]:
def get_company_website_from_wiki(wiki_page: str) -> str:
    try:
        wiki_response = requests.get(wiki_page)
        company_soup = BeautifulSoup(wiki_response.text, 'html.parser')
        website_tag = company_soup.find('span', {'class':'official-website'})
        return website_tag.find('a')['href']
    except:
        return None
    
threeM = "https://en.wikipedia.org/wiki/3M"
get_company_website_from_wiki(threeM)

'https://www.3m.com/'

In [126]:
df_official_website = df.copy(deep=True)
df_official_website.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,https://en.wikipedia.org/wiki/3M
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,https://en.wikipedia.org/wiki/Abiomed


In [127]:
df_official_website["Official Website"] = df_official_website.apply(lambda row: get_company_website_from_wiki(row["Wiki Link"]), axis=1)

In [128]:
df_official_website.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,https://en.wikipedia.org/wiki/3M,https://www.3m.com/
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith,None
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories,https://www.abbott.com
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie,https://www.abbvie.com
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,https://en.wikipedia.org/wiki/Abiomed,https://www.abiomed.com


In [129]:
len(df_official_website[df_official_website["Official Website"].isnull()])

140

Looks like we have 140 companies for which the first method of extracting their official website did not work.

#### Getting Official Website from Wiki Page: Infobox Method

Doing this since there were quite a few wiki pages for which the first method didn't work.

1. Get the soup for the company wiki page
2. Search for the table `table` tag with a class "infobox vcard"
3. Find the table row that has the text "Website"
4. Find the `a` tag within that row and get the href associated with it.

Example on A.O. Smith (a company for which the other method didn't work)

In [130]:
aosmith = "https://en.wikipedia.org/wiki/A._O._Smith"
response = requests.get(aosmith)
print(response.status_code)

aosmith_soup = BeautifulSoup(response.text)
info_table=aosmith_soup.find('table',{'class':"infobox vcard"})

200


In [131]:
rows = info_table.findAll('tr')

for row in rows:
    header = row.find('th')
    if header and header.contents[0] == "Website":
        website = row.find('a')['href']
        break
    
print(website)

http://aosmith.com


Make a function that does this.

In [132]:
def get_company_website_from_infobox(wiki_page: str) -> str:
    if not wiki_page:
        return None
    
    response = requests.get(wiki_page)
    soup = BeautifulSoup(response.text)
    info_table=soup.find('table',{'class':"infobox vcard"})
    rows = info_table.findAll('tr')

    for row in rows:
        header = row.find('th')
        if header and header.contents[0] == "Website":
            website = row.find('a')['href']
            return website

    return None

In [133]:
df_infobox = df.copy(deep=True)
df_infobox["Official Website"] = df_infobox.apply(lambda row: get_company_website_from_infobox(row["Wiki Link"]), axis=1)

In [134]:
df_infobox.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,https://en.wikipedia.org/wiki/3M,https://www.3m.com/
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith,http://aosmith.com
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories,http://www.abbott.com
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie,http://abbvie.com
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,https://en.wikipedia.org/wiki/Abiomed,http://abiomed.com


In [135]:
len(df_infobox[df_infobox["Official Website"].isnull()])


12

This is a bit better than the other method. There are only 12 companies for which the method did not succeed. View them below.

In [136]:
df_infobox[df_infobox["Official Website"].isnull()]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website
23,GOOGL,Alphabet Inc. (Class A),reports,Communication Services,Interactive Media & Services,"Mountain View, California",2014-04-03,1652044,1998,None,None
24,GOOG,Alphabet Inc. (Class C),reports,Communication Services,Interactive Media & Services,"Mountain View, California",2006-04-03,1652044,1998,None,None
156,DOV,Dover Corporation,reports,Industrials,Industrial Machinery,"Downers Grove, Illinois",1985-10-31,29905,1955,https://en.wikipedia.org/wiki/Dover_Corporation,None
205,FTV,Fortive,reports,Industrials,Industrial Machinery,"Everett, Washington",2016-07-01,1659166,2016,https://en.wikipedia.org/wiki/Fortive,None
207,FOXA,Fox Corporation (Class A),reports,Communication Services,Movies & Entertainment,"New York City, New York",2013-07-01,1754301,2019,None,None
208,FOX,Fox Corporation (Class B),reports,Communication Services,Movies & Entertainment,"New York City, New York",2015-09-18,1754301,2019,None,None
219,GILD,Gilead Sciences,reports,Health Care,Biotechnology,"Foster City, California",2004-07-01,882095,1987,https://en.wikipedia.org/wiki/Gilead_Sciences,None
331,NFLX,Netflix,reports,Communication Services,Movies & Entertainment,"Los Gatos, California",2010-12-20,1065280,1997,https://en.wikipedia.org/wiki/Netflix,None
334,NWSA,News Corp (Class A),reports,Communication Services,Publishing,"New York City, New York",2013-08-01,1564708,2013 (1980),None,None
335,NWS,News Corp (Class B),reports,Communication Services,Publishing,"New York City, New York",2015-09-18,1564708,2013 (1980),None,None


#### Getting Twitter Handles from Company Websites: Method 1

1. Get the soup for the company website
2. Search for an `a` tag with an `href` that matches the Twitter link regex `(https:\/\/twitter.com\/(?![a-zA-Z0-9_]+\/)([a-zA-Z0-9_]+))`. That will be the Twitter URL.
3. Split the Twitter URL at backslashes, and take the part after the last backslash as the Twitter handle.

Example on A.O. Smith page

In [137]:
aosmith = "http://aosmith.com"
response = requests.get(aosmith)
print(response.status_code)

aosmith_soup = BeautifulSoup(response.text)

TWITTER_HANDLE_LINK_REGEX = '(https:\/\/twitter.com\/(?![a-zA-Z0-9_]+\/)([a-zA-Z0-9_]+))'
twitter_link_tag = aosmith_soup.find('a', attrs={'href': re.compile(TWITTER_HANDLE_LINK_REGEX)})
print(twitter_link_tag['href'])
print(twitter_link_tag['href'].split("/")[-1])

200
https://twitter.com/aosmithhotwater
aosmithhotwater


#### Getting Twitter Handles from Company Websites: Method 2

1. Get the soup for the company website
2. Search for a single `a` tag with an `href` that contains the string `twitter`.
3. Split that URL at `?` which indicates the start of query params, and take the part before `?` as the Twitter URL.
4. Split the Twitter URL at backslashes, and take the part after the last backslash as the Twitter handle.

Example on Activision Blizzard page (which has its Twitter URL listed as [https://twitter.com/atvi_ab?lang=en](https://twitter.com/atvi_ab?lang=en))

In [138]:
activisionblizzard = "https://activisionblizzard.com/"
response = requests.get(activisionblizzard)
print(response.status_code)

activisionblizzard_soup = BeautifulSoup(response.text)

twitter_link_tag = activisionblizzard_soup.select_one("a[href*=twitter]")
twitter_url = twitter_link_tag['href']
twitter_url_no_params = twitter_link_tag['href'].split("?")[0]
twitter_handle = twitter_url_no_params.split("/")[-1]
print(twitter_url_no_params)
print(twitter_handle)

200
https://twitter.com/atvi_ab
atvi_ab


Make this a function!

In [2]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create a new requests session that allows 0 retries 
session = requests.Session()
retry = Retry(total=0)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# User-Agent header is required to successfully request some sites
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# Map each company's Twitter URL to its corresponding Twitter handle
twitter_url_to_handle_map = {}

def get_twitter_from_website(website: str, mergr: bool = False) -> str:
    """
    Returns company Twitter URL.
    Also updates Twitter URL to Twitter handle map.
    """
    print(website)
    if not website:
        # print("No website")
        return None

    if mergr:
        website = 'https://' + website
            
    try:
        response = session.get(website, headers=HEADERS)
    except:
        print("Request failed")
        return None
    
    print("Scraping", website)
    company_soup = BeautifulSoup(response.text)

    twitter_link_tag = company_soup.select_one("a[href*=twitter]")

    if twitter_link_tag and twitter_link_tag['href']:
        twitter_url = twitter_link_tag['href']
        twitter_url_no_params = twitter_link_tag['href'].split("?")[0]
        twitter_handle = twitter_url_no_params.split("/")[-1]
        twitter_url_to_handle_map[twitter_url_no_params] = twitter_handle
        # print(twitter_url_no_params)
        # print(twitter_handle)
        return twitter_url_no_params
        
    print("Request succeeded but Twitter not found")
    return None

In [139]:
# Try this on 3M
threem = "https://www.3m.com/"
get_twitter_from_website(threem)

'https://twitter.com/3M'

Add the Twitter URLs to the dataframe.

In [140]:
df_twitter = df_infobox.copy(deep=True)
df_twitter["Twitter URL"] = df_twitter.apply(lambda row: get_twitter_from_website(row["Official Website"]), axis=1)

df_twitter

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website,Twitter URL
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,https://en.wikipedia.org/wiki/3M,https://www.3m.com/,https://twitter.com/3M
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith,http://aosmith.com,https://twitter.com/aosmithhotwater
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories,http://www.abbott.com,https://twitter.com/AbbottNews
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie,http://abbvie.com,https://twitter.com/abbvie
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,https://en.wikipedia.org/wiki/Abiomed,http://abiomed.com,https://twitter.com/abiomed
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,https://en.wikipedia.org/wiki/Yum!_Brands,https://www.yum.com,https://twitter.com/kfc
498,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,https://en.wikipedia.org/wiki/Zebra_Technologies,http://www.zebra.com,http://www.twitter.com/zebratechnology
499,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,https://en.wikipedia.org/wiki/Zimmer_Biomet,http://zimmerbiomet.com,https://twitter.com/zimmerbiomet
500,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873,https://en.wikipedia.org/wiki/Zions_Bancorpora...,http://zionsbancorporation.com,None


Use the Twitter URL to Twitter handle map to add Twitter handles to the dataframe.

In [3]:
def get_handle_from_twitter_url(row):
    try:
        return twitter_url_to_handle_map[row["Twitter URL"]]
    except:
        return None

In [141]:
df_twitter["Twitter Handle"] = df_twitter.apply(lambda row: get_handle_from_twitter_url(row), axis=1)
df_twitter

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website,Twitter URL,Twitter Handle
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902,https://en.wikipedia.org/wiki/3M,https://www.3m.com/,https://twitter.com/3M,3M
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith,http://aosmith.com,https://twitter.com/aosmithhotwater,aosmithhotwater
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories,http://www.abbott.com,https://twitter.com/AbbottNews,AbbottNews
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie,http://abbvie.com,https://twitter.com/abbvie,abbvie
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981,https://en.wikipedia.org/wiki/Abiomed,http://abiomed.com,https://twitter.com/abiomed,abiomed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,https://en.wikipedia.org/wiki/Yum!_Brands,https://www.yum.com,https://twitter.com/kfc,kfc
498,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,https://en.wikipedia.org/wiki/Zebra_Technologies,http://www.zebra.com,http://www.twitter.com/zebratechnology,zebratechnology
499,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,https://en.wikipedia.org/wiki/Zimmer_Biomet,http://zimmerbiomet.com,https://twitter.com/zimmerbiomet,zimmerbiomet
500,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873,https://en.wikipedia.org/wiki/Zions_Bancorpora...,http://zionsbancorporation.com,None,None


In [142]:
len(df_twitter[df_twitter["Twitter URL"].isnull()])

165

Looks like we have 165 companies for which we weren't able to extract Twitter URLs. Let's see what those companies are.

In [143]:
df_twitter[df_twitter["Twitter URL"].isnull()]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website,Twitter URL,Twitter Handle
7,ADM,ADM,reports,Consumer Staples,Agricultural Products,"Chicago, Illinois",1981-07-29,7084,1902,https://en.wikipedia.org/wiki/ADM_(company),http://adm.com,None,None
8,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982,https://en.wikipedia.org/wiki/Adobe_Inc.,http://adobe.com,None,None
16,ALK,Alaska Air Group,reports,Industrials,Airlines,"SeaTac, Washington",2016-05-13,766421,1985,https://en.wikipedia.org/wiki/Alaska_Air_Group,http://alaskaair.com,None,None
18,ARE,Alexandria Real Estate Equities,reports,Real Estate,Office REITs,"Pasadena, California",2017-03-20,1035443,1994,https://en.wikipedia.org/wiki/Alexandria_Real_...,https://www.are.com/,None,None
23,GOOGL,Alphabet Inc. (Class A),reports,Communication Services,Interactive Media & Services,"Mountain View, California",2014-04-03,1652044,1998,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,WM,Waste Management,reports,Industrials,Environmental & Facilities Services,"Houston, Texas",NaN,823768,1968,https://en.wikipedia.org/wiki/Waste_Management...,https://www.wm.com,None,None
489,WY,Weyerhaeuser,reports,Real Estate,Specialized REITs,"Seattle, Washington",NaN,106535,1900,https://en.wikipedia.org/wiki/Weyerhaeuser,https://www.weyerhaeuser.com/,None,None
494,WYNN,Wynn Resorts,reports,Consumer Discretionary,Casinos & Gaming,"Paradise, Nevada",2008-11-14,1174922,2002,https://en.wikipedia.org/wiki/Wynn_Resorts,http://wynnresorts.com,None,None
495,XEL,Xcel Energy,reports,Utilities,Multi-Utilities,"Minneapolis, Minnesota",1957-03-04,72903,1909,https://en.wikipedia.org/wiki/Xcel_Energy,https://www.xcelenergy.com,None,None


In [144]:
len(df_twitter[df_twitter["Twitter Handle"].isnull()])

165

Looks like we have 165 companies for which we weren't able to map Twitter Handles. Let's see what those companies are.

In [145]:
df_twitter[df_twitter["Twitter Handle"].isnull()]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website,Twitter URL,Twitter Handle
7,ADM,ADM,reports,Consumer Staples,Agricultural Products,"Chicago, Illinois",1981-07-29,7084,1902,https://en.wikipedia.org/wiki/ADM_(company),http://adm.com,None,None
8,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982,https://en.wikipedia.org/wiki/Adobe_Inc.,http://adobe.com,None,None
16,ALK,Alaska Air Group,reports,Industrials,Airlines,"SeaTac, Washington",2016-05-13,766421,1985,https://en.wikipedia.org/wiki/Alaska_Air_Group,http://alaskaair.com,None,None
18,ARE,Alexandria Real Estate Equities,reports,Real Estate,Office REITs,"Pasadena, California",2017-03-20,1035443,1994,https://en.wikipedia.org/wiki/Alexandria_Real_...,https://www.are.com/,None,None
23,GOOGL,Alphabet Inc. (Class A),reports,Communication Services,Interactive Media & Services,"Mountain View, California",2014-04-03,1652044,1998,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,WM,Waste Management,reports,Industrials,Environmental & Facilities Services,"Houston, Texas",NaN,823768,1968,https://en.wikipedia.org/wiki/Waste_Management...,https://www.wm.com,None,None
489,WY,Weyerhaeuser,reports,Real Estate,Specialized REITs,"Seattle, Washington",NaN,106535,1900,https://en.wikipedia.org/wiki/Weyerhaeuser,https://www.weyerhaeuser.com/,None,None
494,WYNN,Wynn Resorts,reports,Consumer Discretionary,Casinos & Gaming,"Paradise, Nevada",2008-11-14,1174922,2002,https://en.wikipedia.org/wiki/Wynn_Resorts,http://wynnresorts.com,None,None
495,XEL,Xcel Energy,reports,Utilities,Multi-Utilities,"Minneapolis, Minnesota",1957-03-04,72903,1909,https://en.wikipedia.org/wiki/Xcel_Energy,https://www.xcelenergy.com,None,None


**Save Twitter dataframe as a CSV!**

In [146]:
df_twitter.to_csv("sp_500_twitter.csv")

#### Manual Corrections and Additions

Manually go through each row of the `sp_500_twitter.csv` to correct any erroneous Twitter URL or Twitter handle cells, and to fill empty official website, Twitter URL, and Twitter handle cells. Save the new CSV as `sp_500_twitter_manual.csv`

#### Getting Subsidiary Twitter Handles from Official Website

What should we do about S&P 500 companies that are parent companies with quite a few subsidiaries / divisions / brands?

We decided to create a new row for each subsidiary listed on the homepage of the parent company's official website. We'll skip subsidiaries that aren't listed that way for now — it'll help us rule out companies that aren't public-facing using a company-decided variable.

1. Get the soup for the company website
2. Search for all `a` tags with an `href` that contains the string `twitter`.
3. Clean up each Twitter URL and its corresponding handle.
5. Put each (Twitter URL, Twitter handle) tuple into a set for the company.

Example on Yum! Brands page (which has the Twitter URLs of four subsidiaries (Habit Burger, KFC, Pizza Hut, Taco Bell) listed as on its homepage).

In [2]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create a new requests session that allows 0 retries 
session = requests.Session()
retry = Retry(total=0)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# User-Agent header is required to successfully request some sites
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

yum = "https://www.yum.com"
response = session.get(yum, headers=HEADERS, timeout=10)
print(response.status_code)

yum_soup = BeautifulSoup(response.text)

twitter_link_tags = yum_soup.select("a[href*=twitter]")

twitter_url_handle_tuples = set()
for twitter_link_tag in twitter_link_tags:
    twitter_url = twitter_link_tag['href']
    twitter_url_no_params = twitter_link_tag['href'].split("?")[0]
    twitter_handle = twitter_url_no_params.split("/")[-1]
    twitter_url_end = twitter_url_no_params.split(".com/")[-1]
    twitter_handle = twitter_url_end.split("/")[0]
    twitter_url_handle_tuples.add((twitter_url_no_params, twitter_handle))

print(twitter_url_handle_tuples)

200
{('https://twitter.com/habitburger', 'habitburger'), ('https://twitter.com/tacobell', 'tacobell'), ('https://twitter.com/kfc', 'kfc'), ('https://twitter.com/yumbrands', 'yumbrands'), ('https://twitter.com/pizzahut', 'pizzahut')}


Make this a function!

In [3]:
# Map each company's symbol to a set of its and its subsidiaries' (Twitter URL, Twitter handle) tuples
symbol_to_twitters_map = {}

In [3]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create a new requests session that allows 0 retries 
session = requests.Session()
retry = Retry(total=0)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# User-Agent header is required to successfully request some sites
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

def get_twitters_from_website(symbol: str, website: str) -> str:
    """
    Returns a set of all unique (Twitter URL, Twitter handle) tuples scraped from company website,
    with no repeated Twitter handles.
    Also updates symbol to Twitter URLs map.
    """
    if not website:
        # print("No website")
        return None
        
    try:
        response = session.get(website, headers=HEADERS, timeout=10)
    except:
        # print("Request failed")
        return None
    
    # print("Scraping", website)
    company_soup = BeautifulSoup(response.text)
    
    twitter_link_tags = company_soup.select("a[href*=twitter]")

    twitter_handles = set()
    twitter_url_handle_tuples = set()
    for twitter_link_tag in twitter_link_tags:
        if twitter_link_tag['href']:
            twitter_url = twitter_link_tag['href']
            twitter_url_no_params = twitter_link_tag['href'].split("?")[0]
            twitter_url_end = twitter_url_no_params.split(".com/")[-1]
            twitter_handle = twitter_url_end.split("/")[0]
            if twitter_handle not in twitter_handles and twitter_handle != "intent":
                # We don't want Twitter handles that say "intent" -> that's usually something else
                twitter_url_handle_tuples.add((twitter_url_no_params, twitter_handle))
                twitter_handles.add(twitter_handle)
    
    symbol_to_twitters_map[symbol] = twitter_url_handle_tuples
    return(twitter_url_handle_tuples)
        
# Try this on Yum
# yum = "https://www.yum.com"
# print(get_twitters_from_website("YUM", yum))
# print()
# print(symbol_to_twitters_map)

Read in our manually updated CSV with company website links and Twitter URLs.

Add a Subsidiary column, set to False for all rows since we only have parent companies for now.

In [5]:
df_manual = pd.read_csv('sp_500_twitter_manual.csv')
df_manual["Subsidiary"]=False
df_manual

,Unnamed: 0,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website,Twitter URL,Twitter Handle,Subsidiary
0,0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/76,66740,1902,https://en.wikipedia.org/wiki/3M,https://www.3m.com/,https://twitter.com/3M,3M,False
1,1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",7/26/17,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith,http://aosmith.com,https://twitter.com/aosmithhotwater,aosmithhotwater,False
2,2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/64,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories,http://www.abbott.com,https://twitter.com/AbbottNews,AbbottNews,False
3,3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/12,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie,http://abbvie.com,https://twitter.com/abbvie,abbvie,False
4,4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/18,815094,1981,https://en.wikipedia.org/wiki/Abiomed,http://abiomed.com,https://twitter.com/abiomed,abiomed,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,497,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",10/6/97,1041061,1997,https://en.wikipedia.org/wiki/Yum!_Brands,https://www.yum.com,https://twitter.com/yumbrands,yumbrands,False
498,498,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",12/23/19,877212,1969,https://en.wikipedia.org/wiki/Zebra_Technologies,http://www.zebra.com,http://www.twitter.com/zebratechnology,zebratechnology,False
499,499,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",8/7/01,1136869,1927,https://en.wikipedia.org/wiki/Zimmer_Biomet,http://zimmerbiomet.com,https://twitter.com/zimmerbiomet,zimmerbiomet,False
500,500,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",6/22/01,109380,1873,https://en.wikipedia.org/wiki/Zions_Bancorpora...,http://zionsbancorporation.com,NaN,NaN,False


In [6]:
# For each row of dataframe, find all Twitter URLs on company official website
# Save in dictionary mapping from company Symbol to set of (Twitter URL, Twitter handle) tuples found on that company's official website
for row in df_manual.itertuples():
    get_twitters_from_website(row.Symbol, row._12) # _12 corresponds to the dataframe's Official Website column
    # print(row.Symbol)
    
# symbol_to_twitters_map

/Users/michellelum/anaconda3/envs/econtext/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Now it's time to add a new row for each subsidiary's Twitter account to our dataframe!
Let's make a copy of our dataframe.

In [7]:
import warnings
warnings.filterwarnings('ignore') # ignore warnings about append

df_manual_subsidiaries = df_manual.copy(deep=True)

for symbol in symbol_to_twitters_map:
    if symbol == "TWTR":
        # Twitter has a bunch of Twitter URLs on its website that aren't subsidiaries -> skip Twitter
        continue
        
    twitter_tuples = symbol_to_twitters_map[symbol]
    num_twitters_scraped = len(twitter_tuples)
    
    # If a single Twitter account was scraped, that's probably just the parent company's Twitter account,
    # which is already in our dataframe, so we don't want to add a new row for that.
    if num_twitters_scraped > 1:
        # Get index of the parent company's row by finding the first row that matches the company Symbol
        parent_company_row_index = df_manual.loc[df_manual["Symbol"] == symbol].index[0]
        
        # Copy that old parent company row to a new subsidiary row
        parent_company_row = df_manual.loc[parent_company_row_index]
        subsidiary_row = parent_company_row.copy()
        
        # Iterate thru the (Twitter URL, Twitter Handle) tuples for this parent company
        for twitter_url, twitter_handle in twitter_tuples:
            if twitter_url != parent_company_row["Twitter URL"] and twitter_handle.lower() != parent_company_row["Twitter Handle"].lower():
                # It's a subsidiary Twitter URL, not the parent Twitter URL!
                # Update the Twitter URL, Twitter Handle, and Subsidiary columns for the new row
                subsidiary_row["Twitter URL"] = twitter_url
                subsidiary_row["Twitter Handle"] = twitter_handle
                subsidiary_row["Subsidiary"] = True
                        
                # Add subsidiary as new row in df
                df_manual_subsidiaries = df_manual_subsidiaries.append(subsidiary_row)

In [11]:
# Sort dataframe alphabetically, primarily by company Security, and secondarily by Subsidiary column value
# The lambda function ensures that the sort is case-insensitive
# So that parent companies and their subsidiaries are in consecutive rows
df_manual_subsidiaries["Subsidiary"] = df_manual_subsidiaries["Subsidiary"].map({True: 'True', False: 'False'}) # convert Subsidiary column to strings to do the sorting
df_manual_subsidiaries = df_manual_subsidiaries.sort_values(["Security", "Subsidiary"], key=lambda col: col.str.lower())
df_manual_subsidiaries["Subsidiary"] = df_manual_subsidiaries["Subsidiary"].map({'True': True, 'False': False}) # convert Subsidiary column back to bools

# Save as CSV
df_manual_subsidiaries.to_csv("sp_500_twitter_subsidiaries.csv")

In [12]:
# Subtract 1 because 1 row is the column headings
print("Number of rows in original df: ", len(df_manual) - 1)
print("Number of rows in new df: ", len(df_manual_subsidiaries) - 1)

Number of rows in original df:  501
Number of rows in new df:  558


Looks like we found 57 new Twitter accounts! Does this mean 57 potential subsidiaries?

We may want to manually check to ensure that these aren't people (ex: CEOs) or unrelated Twitter accounts

In [3]:
df_subsidiaries_actually_manual = pd.read_csv('sp_500_twitter_subsidiaries_manual.csv')
print("Number of rows in manually corrected df: ", len(df_subsidiaries_actually_manual) - 1)

Number of rows in manually corrected df:  527


From our manual corrections, it looks like we actually found just 26 new subsidiaries with Twitter accounts. Still not bad!

#### Drop Rows with Duplicate Twitter Handles

To avoid replicating the same tweets in our dataset!

In [12]:
# Read in our manually updated CSV with subsidiary Twitters. Make a copy of this CSV!
df_with_duplicates = pd.read_csv('sp_500_twitter_subsidiaries_manual.csv')
df_no_duplicates = df_with_duplicates.copy(deep=True)

In [13]:
# Drop duplicates (rows that have the same Twitter Handle values) except for the first occurrence,
# and don't drop any rows that have a null value for the Twitter Handle column.
df_no_duplicates = df_with_duplicates[(~df_with_duplicates['Twitter Handle'].duplicated()) | df_with_duplicates['Twitter Handle'].isna()]

# df_no_duplicates = df_no_duplicates.drop_duplicates(subset='Twitter Handle')

df_no_duplicates

,Unnamed: 0.1,Unnamed: 0,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Wiki Link,Official Website,Twitter URL,Twitter Handle,Subsidiary
0,0,0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/76,66740,1902,https://en.wikipedia.org/wiki/3M,https://www.3m.com/,https://twitter.com/3M,3M,False
1,1,1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",7/26/17,91142,1916,https://en.wikipedia.org/wiki/A._O._Smith,http://aosmith.com,https://twitter.com/aosmithhotwater,aosmithhotwater,False
2,2,2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/64,1800,1888,https://en.wikipedia.org/wiki/Abbott_Laboratories,http://www.abbott.com,https://twitter.com/AbbottNews,AbbottNews,False
3,3,3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/12,1551152,2013 (1888),https://en.wikipedia.org/wiki/AbbVie,http://abbvie.com,https://twitter.com/abbvie,abbvie,False
4,4,4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/18,815094,1981,https://en.wikipedia.org/wiki/Abiomed,http://abiomed.com,https://twitter.com/abiomed,abiomed,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,497,497,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",10/6/97,1041061,1997,https://en.wikipedia.org/wiki/Yum!_Brands,https://www.yum.com,https://twitter.com/kfc,kfc,True
529,498,498,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",12/23/19,877212,1969,https://en.wikipedia.org/wiki/Zebra_Technologies,http://www.zebra.com,http://www.twitter.com/zebratechnology,zebratechnology,False
530,499,499,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",8/7/01,1136869,1927,https://en.wikipedia.org/wiki/Zimmer_Biomet,http://zimmerbiomet.com,https://twitter.com/zimmerbiomet,zimmerbiomet,False
531,500,500,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",6/22/01,109380,1873,https://en.wikipedia.org/wiki/Zions_Bancorpora...,http://zionsbancorporation.com,NaN,NaN,False


In [14]:
# Save as CSV
df_no_duplicates.to_csv("sp_500_twitter_subsidiaries_manual_no_duplicates.csv")

In [15]:
# Subtract 1 because 1 row is the column headings
print("Number of rows in original df: ", len(df_with_duplicates) - 1)
print("Number of rows in new df: ", len(df_no_duplicates) - 1)

Number of rows in original df:  532
Number of rows in new df:  531


It looks like we only dropped 1 row with duplicate Twitter Handles! That was Google, since it has Class A and Class B stocks that correspond to the same Twitter handle.

#### Getting Subsidiary Twitter Handles from Twitter Profile Mentions

We decided to create a new row for each subsidiary listed on an S&P 500 company's Twitter profile. We excluded Twitter accounts that are for customer service / care (e.g. ask, help, or support Twitter accounts), Twitter accounts that are for specific teams within the company (e.g. engineering), Twitter accounts for countries outside the U.S., and Twitter accounts mentioned that aren't actually subsidiaries (e.g. organizations that the company sponsors but does not own).

1. Manually grab subsidiaries from the `mentions` column of `data/sp_500_twitter_profile_info.csv`. Map each original company Twitter handle from `data/sp_500_twitter_profile_info.csv` to a list of Twitter handles of the Twitter accounts mentioned in the original company's Twitter profile.

In [2]:
df_no_duplicates = pd.read_csv('sp_500_twitter_subsidiaries_manual_no_duplicates.csv')

In [3]:
# Map each original company Twitter handle to a list of its and its mentioned subsidiaries' Twitter handles
handle_to_subsidiary_mentions_map = {'ATVI_AB': ['activision', 'blizzard_ent', 'king_games'],
                                     'AlignTechInc': ['Invisalign', 'exocad', 'iTeroScanner'],
                                     'amazon': ['AmazonNews'],
                                     'BioRadFlowAbs': ['BioRadCellBio'],
                                     'BookingHoldings': ['bookingcom', 'kayak', 'priceline', 'opentable', 'agoda'],
                                     'Campbells': ['CampbellSoupCo'],
                                     'CharterNewsroom': ['SpectrumBiz'],
                                     'ChubbNA': ['Chubb'],
                                     'Cisco': ['HeyCisco'],
                                     'edisonintl': ['SCE', 'Edison_Energy'],
                                     'ExpediaGroup': ['expedia', 'travelocity', 'orbitz', 'hotelsdotcom', 'hotwire', 'vrbo', 'expediamedia'],
                                     'FBHS_News': ['Moen', 'ThermaTru', 'FiberonDecking', 'LarsonDoors', 'MasterLockUS'],
                                     'Intuit': ['TurboTax', 'CreditKarma', 'QuickBooks', 'Mailchimp'],
                                     'roberthalf': ['Protiviti'],
                                     'NewsfromRCgroup': ['RoyalCaribbean', 'CelebrityCruise', 'Silversea', 'TUICruises', 'hlcruises'],
                                     'ServiceNow': ['ServiceNowNews']}

Now it's time to add a new row for each mentioned subsidiary's Twitter account to our dataframe! Let's make a copy of our dataframe.

In [4]:
import warnings
warnings.filterwarnings('ignore') # ignore warnings about append

df_subsidiaries_manual_mentioned = df_no_duplicates.copy(deep=True)

mentioned_subsidiaries_count = 0
for handle in handle_to_subsidiary_mentions_map:  
    twitter_handles_mentioned = handle_to_subsidiary_mentions_map[handle]

    # Get index of the parent company's row by finding the first row that matches the company Twitter Handle
    parent_company_row_index = df_no_duplicates.loc[df_no_duplicates["Twitter Handle"].str.lower() == handle.lower()].index[0]

    # Copy that old parent company row to a new subsidiary row
    parent_company_row = df_no_duplicates.loc[parent_company_row_index]
    subsidiary_row = parent_company_row.copy()

    # Iterate thru the Twitter Handles mentioned in this parent company's Twitter profile
    for twitter_handle in twitter_handles_mentioned:
        mentioned_subsidiaries_count += 1
        # Update the Twitter URL, Twitter Handle, and Subsidiary columns for the new row
        subsidiary_row["Twitter URL"] = "https://twitter.com" + twitter_handle
        subsidiary_row["Twitter Handle"] = twitter_handle
        subsidiary_row["Subsidiary"] = True

        # Add subsidiary as new row in df
        df_subsidiaries_manual_mentioned = df_subsidiaries_manual_mentioned.append(subsidiary_row)

print("Number of mentioned subsidiaries:", mentioned_subsidiaries_count)

Number of mentioned subsidiaries: 42


In [5]:
# Sort dataframe alphabetically, primarily by company Security, and secondarily by Subsidiary column value
# The lambda function ensures that the sort is case-insensitive
# So that parent companies and their subsidiaries are in consecutive rows
df_subsidiaries_manual_mentioned["Subsidiary"] = df_subsidiaries_manual_mentioned["Subsidiary"].map({True: 'True', False: 'False'}) # convert Subsidiary column to strings to do the sorting
df_subsidiaries_manual_mentioned = df_subsidiaries_manual_mentioned.sort_values(["Security", "Subsidiary"], key=lambda col: col.str.lower())
df_subsidiaries_manual_mentioned["Subsidiary"] = df_subsidiaries_manual_mentioned["Subsidiary"].map({'True': True, 'False': False}) # convert Subsidiary column back to bools

In [6]:
# Subtract 1 because 1 row is the column headings
print("Number of rows in original df: ", len(df_no_duplicates) - 1)
print("Number of rows in new df: ", len(df_subsidiaries_manual_mentioned) - 1)

Number of rows in original df:  531
Number of rows in new df:  573


Looks like we found 43 new Twitter accounts from mentions in parent company Twitter profiles!

Let's drop any rows with duplicate Twitter handles, in case there are any.

In [7]:
# Drop duplicates (rows that have the same Twitter Handle values) except for the first occurrence,
# and don't drop any rows that have a null value for the Twitter Handle column.
df_subsidiaries_manual_mentioned = df_subsidiaries_manual_mentioned[(~df_subsidiaries_manual_mentioned['Twitter Handle'].duplicated()) | df_subsidiaries_manual_mentioned['Twitter Handle'].isna()]

print("Number of rows in de-duplicated df: ", len(df_subsidiaries_manual_mentioned) - 1)

Number of rows in de-duplicated df:  573


In [8]:
# Save as CSV
df_subsidiaries_manual_mentioned.to_csv("sp_500_twitter_subsidiaries_manual_mentioned.csv")

#### Getting Twitter Handles from Mergr

Using a CSV of the first 250 U.S. public companies, sorted by revenue.

In [4]:
# Read CSV of Mergr companies
df_mergr = pd.read_csv('mergr_11-21-22_by_revenue.csv')
print(len(df_mergr))

250


Remove the Mergr df rows for companies already in our S&P 500 CSV. Note that this may not catch all duplicate companies, since their names may be listed slightly differently on the S&P 500 CSV and the Mergr CSV.

In [5]:
df_sp_500 = pd.read_csv('sp_500_twitter_subsidiaries_manual_mentioned.csv')
sp_500_company_names = df_sp_500['Security']

In [6]:
df_mergr = df_mergr[~df_mergr['Name'].isin(sp_500_company_names)]
print(len(df_mergr))

131


Looks like, based on direct name matching, 131 of the 250 Mergr companies are not already in the S&P 500 CSV.

Add Twitter URLs to the dataframe.

In [7]:
# Get Twitter URLs from Mergr company websites.
df_mergr_twitter = df_mergr.copy(deep=True)
df_mergr_twitter['Twitter URL'] = df_mergr_twitter.apply(lambda row: get_twitter_from_website(row['Url'], mergr=True), axis=1)
df_mergr_twitter

www.apple.com
Scraping https://www.apple.com
Request succeeded but Twitter not found
www.corporate.exxonmobil.com
Request failed
www.abc.xyz
Scraping https://www.abc.xyz
Request succeeded but Twitter not found
www.unitedhealthgroup.com
Scraping https://www.unitedhealthgroup.com
www.costco.com
Scraping https://www.costco.com
Request succeeded but Twitter not found
www.walgreensbootsalliance.com
Scraping https://www.walgreensbootsalliance.com
www.ford.com
Scraping https://www.ford.com
www.homedepot.com
Scraping https://www.homedepot.com
www.centene.com
Scraping https://www.centene.com
www.gm.com
Scraping https://www.gm.com
Request succeeded but Twitter not found
www.marathonpetroleum.com
Scraping https://www.marathonpetroleum.com
Request succeeded but Twitter not found
www.facebook.com
Scraping https://www.facebook.com
Request succeeded but Twitter not found
www.delltechnologies.com
Scraping https://www.delltechnologies.com
www.ups.com
Scraping https://www.ups.com
Request succeeded but T

Scraping https://www.foxcorporation.com
Request succeeded but Twitter not found
www.carvana.com
Scraping https://www.carvana.com
www.otis.com
Scraping https://www.otis.com


,Name,Sector,Address,City,State or Province,Zip,Country,Phone,Url,Revenue ($mlns),Employees,Ownership Status,Investors/Acquirers,Buy M&A Count - Total,Avg. Buy Count Per Year (Last 5 Yrs),Sell M&A Count - Total,Avg. Sell Count Per Year (Last 5 Yrs),Twitter URL
2,Apple,Technology Hardware,1 Infinite Loop,Cupertino,California,95014,United States,(408) 996-1010,www.apple.com,"365,817",150000,Public,NaN,108.0,3.8,2.0,NaN,None
5,Exxon,Oil/Gas Exploration,5959 Las Colinas Boulevard,Irving,Texas,75039,United States,(972) 940-6000,www.corporate.exxonmobil.com,"276,692",63000,Public,NaN,8.0,NaN,6.0,0.4,None
6,Google,Internet Software and Services,1600 Amphitheatre Parkway,Mountain View,California,94043,United States,(650) 253-0000,www.abc.xyz,"257,637",139995,Public,NaN,221.0,4.0,6.0,0.2,None
7,UnitedHealth,Healthcare Services,UnitedHealth Group Center 9900 Bren Road East,Minnetonka,Minnesota,55343,United States,(952) 936-1300,www.unitedhealthgroup.com,"255,639",330000,Public,NaN,16.0,0.6,2.0,NaN,https://twitter.com/UnitedHealthGrp
12,Costco Wholesale,Retail,999 Lake Dr,Issaquah,Washington,98027,United States,(425) 313-8100,www.costco.com,"166,761",156000,Public,NaN,3.0,0.4,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,LKQ,Automotive,"500 West Madison Street, Suite 2800",Chicago,Illinois,60661,United States,(312) 621-1950,www.lkqcorp.com,"13,089",46000,Public,NaN,44.0,0.6,3.0,0.2,https://twitter.com/LKQCorp
243,Markel,Insurance,4521 Highwoods Parkway,Glen Allen,Virginia,23060,United States,(804) 747-0136,www.markel.com,"12,910",20300,Public,Alliance Holdings,13.0,1.0,NaN,NaN,https://twitter.com/MarkelStyle
244,Fox,Media,1211 Avenue of the Americas,New York,New York,10036,United States,(212) 852-7000,www.foxcorporation.com,"12,909",9000,Public,NaN,4.0,0.8,NaN,NaN,None
245,Carvana,E-Commerce,1930 West Rio Salado Parkway,Tempe,Arizona,85281,United States,(480) 719-8809,www.carvana.com,"12,814",21000,Public,NaN,3.0,0.4,NaN,NaN,https://twitter.com/carvana


Use the Twitter URL to Twitter handle map to add Twitter handles to the dataframe.

In [8]:
df_mergr_twitter['Twitter Handle'] = df_mergr_twitter.apply(lambda row: get_handle_from_twitter_url(row), axis=1)
df_mergr_twitter

,Name,Sector,Address,City,State or Province,Zip,Country,Phone,Url,Revenue ($mlns),Employees,Ownership Status,Investors/Acquirers,Buy M&A Count - Total,Avg. Buy Count Per Year (Last 5 Yrs),Sell M&A Count - Total,Avg. Sell Count Per Year (Last 5 Yrs),Twitter URL,Twitter Handle
2,Apple,Technology Hardware,1 Infinite Loop,Cupertino,California,95014,United States,(408) 996-1010,www.apple.com,"365,817",150000,Public,NaN,108.0,3.8,2.0,NaN,None,None
5,Exxon,Oil/Gas Exploration,5959 Las Colinas Boulevard,Irving,Texas,75039,United States,(972) 940-6000,www.corporate.exxonmobil.com,"276,692",63000,Public,NaN,8.0,NaN,6.0,0.4,None,None
6,Google,Internet Software and Services,1600 Amphitheatre Parkway,Mountain View,California,94043,United States,(650) 253-0000,www.abc.xyz,"257,637",139995,Public,NaN,221.0,4.0,6.0,0.2,None,None
7,UnitedHealth,Healthcare Services,UnitedHealth Group Center 9900 Bren Road East,Minnetonka,Minnesota,55343,United States,(952) 936-1300,www.unitedhealthgroup.com,"255,639",330000,Public,NaN,16.0,0.6,2.0,NaN,https://twitter.com/UnitedHealthGrp,UnitedHealthGrp
12,Costco Wholesale,Retail,999 Lake Dr,Issaquah,Washington,98027,United States,(425) 313-8100,www.costco.com,"166,761",156000,Public,NaN,3.0,0.4,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,LKQ,Automotive,"500 West Madison Street, Suite 2800",Chicago,Illinois,60661,United States,(312) 621-1950,www.lkqcorp.com,"13,089",46000,Public,NaN,44.0,0.6,3.0,0.2,https://twitter.com/LKQCorp,LKQCorp
243,Markel,Insurance,4521 Highwoods Parkway,Glen Allen,Virginia,23060,United States,(804) 747-0136,www.markel.com,"12,910",20300,Public,Alliance Holdings,13.0,1.0,NaN,NaN,https://twitter.com/MarkelStyle,MarkelStyle
244,Fox,Media,1211 Avenue of the Americas,New York,New York,10036,United States,(212) 852-7000,www.foxcorporation.com,"12,909",9000,Public,NaN,4.0,0.8,NaN,NaN,None,None
245,Carvana,E-Commerce,1930 West Rio Salado Parkway,Tempe,Arizona,85281,United States,(480) 719-8809,www.carvana.com,"12,814",21000,Public,NaN,3.0,0.4,NaN,NaN,https://twitter.com/carvana,carvana


Remove the Mergr df rows for Twitter handles already in our S&P 500 CSV. Do this in a case-insensitive way.

In [9]:
sp_500_twitter_handles = df_sp_500['Twitter Handle'].dropna()  # Drop nulls

In [10]:
df_mergr_twitter = df_mergr_twitter[~df_mergr_twitter['Twitter Handle'].str.lower().isin([sp_500_handle.lower() for sp_500_handle in sp_500_twitter_handles])]
print(len(df_mergr_twitter))

85


Looks like, based on direct Twitter Handle matching, 85 of the 250 Mergr companies are either 1) not already in the S&P 500 CSV, or 2) we weren't able to scrape their Twitter handles.

In [11]:
len(df_mergr_twitter[df_mergr_twitter["Twitter URL"].isnull()])

50

Looks like we have 50 companies for which we weren't able to extract Twitter URLs. Let's see what those companies are.

In [12]:
df_mergr_twitter[df_mergr_twitter["Twitter Handle"].isnull()]

,Name,Sector,Address,City,State or Province,Zip,Country,Phone,Url,Revenue ($mlns),Employees,Ownership Status,Investors/Acquirers,Buy M&A Count - Total,Avg. Buy Count Per Year (Last 5 Yrs),Sell M&A Count - Total,Avg. Sell Count Per Year (Last 5 Yrs),Twitter URL,Twitter Handle
2,Apple,Technology Hardware,1 Infinite Loop,Cupertino,California,95014,United States,(408) 996-1010,www.apple.com,"365,817",150000,Public,NaN,108.0,3.8,2.0,NaN,None,None
5,Exxon,Oil/Gas Exploration,5959 Las Colinas Boulevard,Irving,Texas,75039,United States,(972) 940-6000,www.corporate.exxonmobil.com,"276,692",63000,Public,NaN,8.0,NaN,6.0,0.4,None,None
6,Google,Internet Software and Services,1600 Amphitheatre Parkway,Mountain View,California,94043,United States,(650) 253-0000,www.abc.xyz,"257,637",139995,Public,NaN,221.0,4.0,6.0,0.2,None,None
12,Costco Wholesale,Retail,999 Lake Dr,Issaquah,Washington,98027,United States,(425) 313-8100,www.costco.com,"166,761",156000,Public,NaN,3.0,0.4,NaN,NaN,None,None
21,GM,Automotive,300 Renaissance Center,Detroit,Michigan,48265,United States,(313) 667-1500,www.gm.com,"122,485",155000,Public,NaN,4.0,NaN,6.0,0.2,None,None
23,Marathon,Energy Services and Equipment,539 South Main Street,Findlay,Ohio,45840,United States,(419) 422-2121,www.marathonpetroleum.com,"119,983",17700,Public,NaN,2.0,0.2,2.0,0.4,None,None
24,Facebook,Internet Software and Services,1601 Willow Road,Menlo Park,California,94025,United States,(650) 543-4800,www.facebook.com,"117,929",77805,Public,NaN,83.0,3.2,NaN,NaN,None,None
27,UPS,Transportation,"55 Glenlake Parkway, NE",Atlanta,Georgia,30328,United States,(404) 828-6000,www.ups.com,"97,287",273510,Public,NaN,19.0,0.4,4.0,0.4,None,None
31,Target,Retail,1000 Nicollet Mall,Minneapolis,Minnesota,55403,United States,(612) 304-6073,www.targetcorp.com,"93,561",409000,Public,NaN,9.0,0.2,5.0,NaN,None,None
41,Citi,Financial Services,388 Greenwich Street,New York,New York,10013,United States,(212) 559-1000,www.citigroup.com,"71,887",231000,Public,NaN,16.0,NaN,18.0,0.2,None,None


Save Twitter dataframe as a CSV!

In [13]:
df_mergr_twitter.to_csv("mergr_twitter.csv")

#### Mergr Manual Corrections and Additions

Manually go through each row of the `mergr_twitter.csv` to correct any erroneous Twitter URL or Twitter handle cells, and to fill empty Twitter URL and Twitter handle cells. Save the new CSV as `mergr_twitter_manual.csv`

In [3]:
# Read CSV of Mergr companies with Twitter handles
df_mergr_twitter_manual = pd.read_csv('mergr_twitter_manual.csv')
print(len(df_mergr_twitter_manual))

85


Remove the Mergr df rows for Twitter handles already in our S&P 500 CSV. Do this in a case-insensitive way.

In [4]:
df_sp_500 = pd.read_csv('sp_500_twitter_subsidiaries_manual_mentioned.csv')
sp_500_twitter_handles = df_sp_500['Twitter Handle'].dropna()  # Drop nulls

In [5]:
df_mergr_twitter_dedup = df_mergr_twitter_manual[~df_mergr_twitter_manual['Twitter Handle'].str.lower().isin([sp_500_handle.lower() for sp_500_handle in sp_500_twitter_handles])]
print(len(df_mergr_twitter_dedup))

53


Looks like, based on direct Twitter Handle matching, 53 of the 250 Mergr companies are not already in the S&P 500 CSV.

Save dataframe with deduplicated Twitter handles as a CSV!

In [6]:
df_mergr_twitter_dedup.to_csv("mergr_twitter_manual_deduplicated_from_sp_500.csv")

#### Getting Mergr Subsidiary Twitter Handles from Official Website


What should we do about Mergr companies that are parent companies with quite a few subsidiaries / divisions / brands?

We decided to create a new row for each subsidiary listed on the homepage of the parent company's official website. We'll skip subsidiaries that aren't listed that way for now — it'll help us rule out companies that aren't public-facing using a company-decided variable.

1. Get the soup for the company website
2. Search for all `a` tags with an `href` that contains the string `twitter`.
3. Clean up each Twitter URL and its corresponding handle.
4. Put each (Twitter URL, Twitter handle) tuple into a set for the company.

In [7]:
# Map each company's name to a set of its and its subsidiaries' (Twitter URL, Twitter handle) tuples
name_to_twitters_map = {}

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create a new requests session that allows 0 retries 
session = requests.Session()
retry = Retry(total=0)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# User-Agent header is required to successfully request some sites
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

def get_twitters_from_website_mergr(name: str, website: str) -> str:
    """
    Returns a set of all unique (Twitter URL, Twitter handle) tuples scraped from company website,
    with no repeated Twitter handles.
    Also updates name to Twitter URLs map.
    """
    if not website:
        # print("No website")
        return None
    
    website = 'https://' + website
        
    try:
        response = session.get(website, headers=HEADERS, timeout=10)
    except:
        # print("Request failed")
        return None
    
    # print("Scraping", website)
    company_soup = BeautifulSoup(response.text)
    
    twitter_link_tags = company_soup.select("a[href*=twitter]")

    twitter_handles = set()
    twitter_url_handle_tuples = set()
    for twitter_link_tag in twitter_link_tags:
        if twitter_link_tag['href']:
            twitter_url = twitter_link_tag['href']
            twitter_url_no_params = twitter_link_tag['href'].split("?")[0]
            twitter_url_end = twitter_url_no_params.split(".com/")[-1]
            twitter_handle = twitter_url_end.split("/")[0]
            if twitter_handle not in twitter_handles and twitter_handle != "intent":
                # We don't want Twitter handles that say "intent" -> that's usually something else
                twitter_url_handle_tuples.add((twitter_url_no_params, twitter_handle))
                twitter_handles.add(twitter_handle)
    
    name_to_twitters_map[name] = twitter_url_handle_tuples
    return(twitter_url_handle_tuples)

Read in our manually updated CSV with company website links and Twitter URLs.

Add a Subsidiary column, set to False for all rows since we only have parent companies for now.

In [8]:
df_mergr_manual = pd.read_csv('mergr_twitter_manual_deduplicated_from_sp_500.csv')
df_mergr_manual["Subsidiary"]=False
df_mergr_manual

,Unnamed: 0.1,Unnamed: 0,Name,Sector,Address,City,State or Province,Zip,Country,Phone,...,Employees,Ownership Status,Investors/Acquirers,Buy M&A Count - Total,Avg. Buy Count Per Year (Last 5 Yrs),Sell M&A Count - Total,Avg. Sell Count Per Year (Last 5 Yrs),Twitter URL,Twitter Handle,Subsidiary
0,7,24,Facebook,Internet Software and Services,1601 Willow Road,Menlo Park,California,94025,United States,(650) 543-4800,...,77805,Public,NaN,83.0,3.2,NaN,NaN,https://twitter.com/facebook,facebook,False
1,8,26,Dell,Technology Hardware,1 Dell Way,Round Rock,Texas,78682,United States,(800) 289-3355,...,133000,Public,NaN,29.0,NaN,7.0,0.2,https://twitter.com/delltech,delltech,False
2,12,44,Energy Transfer,Energy Services and Equipment,"8111 Westchester Drive, Suite 600",Dallas,Texas,75225,United States,(214) 981-0700,...,12558,Public,NaN,9.0,0.6,1.0,NaN,https://twitter.com/EnergyTransfer,EnergyTransfer,False
3,13,52,Bunge,Agriculture,1391 Timberlake Manor Parkway,St. Louis,Missouri,63017,United States,(314) 292-2000,...,22000,Public,NaN,10.0,NaN,4.0,0.2,https://twitter.com/bunge,bunge,False
4,14,59,StoneX,Business Services,155 East 44th Street Suite 900,New York,New York,10017,United States,(212) 485-3500,...,2946,Public,NaN,17.0,2.4,NaN,NaN,https://twitter.com/StoneX_Official,StoneX_Official,False
5,19,73,Tyson,Food,2200 W. Don Tyson Parkway,Springdale,Arkansas,72762,United States,(479) 290-4000,...,139000,Public,NaN,13.0,0.6,6.0,0.6,https://twitter.com/TysonFoods,TysonFoods,False
6,21,78,TJX,Retail,770 Cochituate Road,Framingham,Massachusetts,1701,United States,(508) 390-1000,...,320000,Public,NaN,5.0,0.2,1.0,NaN,https://twitter.com/tjmaxx,tjmaxx,False
7,22,79,Enterprise Products Partners,Oil/Gas Exploration,"1100 Louisiana Street, 10th Floor",Houston,Texas,77002,United States,(713) 381-6500,...,0,Public,NaN,19.0,0.4,5.0,0.2,NaN,NaN,False
8,23,80,Coca-Cola,Beverages,One Coca-Cola Plaza,Atlanta,Georgia,30313,United States,(404) 676-2121,...,80300,Public,NaN,18.0,0.4,8.0,0.6,https://twitter.com/CocaCola,CocaCola,False
9,24,87,Arrow Electronics,Distribution,9201 East Dry Creek Road,Centennial,Colorado,80112,United States,(303) 824-4000,...,20700,Public,NaN,34.0,0.2,1.0,NaN,https://twitter.com/Arrow_dot_com,Arrow_dot_com,False


In [9]:
# For each row of dataframe, find all Twitter URLs on company official website
# Save in dictionary mapping from company Name to set of (Twitter URL, Twitter handle) tuples found on that company's official website
for row in df_mergr_manual.itertuples():
    get_twitters_from_website_mergr(row.Name, row.Url)
    print(row.Name)
    
name_to_twitters_map

Facebook
Dell
Energy Transfer
Bunge
StoneX
Tyson
TJX
Enterprise Products Partners
Coca-Cola
Arrow Electronics
Jabil
NRG
UNFI
Macy's
PFG
TD SYNNEX
Rite Aid
Plains All American
US Foods
Lithia Motors
AerSale
ManpowerGroup
Penske
AutoNation
World Fuel Services
Kohl's 
JLL
Lear
PACCAR
Kyndryl
PG&E
HF Sinclair
Landos
Tenet Health
Avnet
Stryker
BJ's
XPO
C.H. Robinson Worldwide
Rocket Companies
Fidelity National Financial
Nordstrom
RGA
ITW
Qurate Retail
Wayfair
Berry
GAP
Loews
AECOM
Markel
Fox
Carvana


{'Facebook': set(),
 'Dell': {('//twitter.com/delltech', 'delltech')},
 'Energy Transfer': set(),
 'Bunge': set(),
 'StoneX': {('https://twitter.com/StoneX_Official', 'StoneX_Official')},
 'Tyson': {('https://twitter.com/TysonFoods/', 'TysonFoods')},
 'TJX': set(),
 'Enterprise Products Partners': set(),
 'Coca-Cola': set(),
 'Arrow Electronics': {('https://twitter.com/Arrow_dot_com', 'Arrow_dot_com')},
 'Jabil': {('https://twitter.com/Jabil', 'Jabil')},
 'NRG': set(),
 'UNFI': {('https://twitter.com/#!/UNFI', '#!')},
 "Macy's": {('https://twitter.com/macysnews', 'macysnews')},
 'PFG': set(),
 'TD SYNNEX': set(),
 'Rite Aid': {('http://www.twitter.com/RiteAid', 'RiteAid')},
 'Plains All American': set(),
 'US Foods': {('https://twitter.com/usfoods', 'usfoods')},
 'Lithia Motors': {('https://www.twitter.com/lithiamotors', 'lithiamotors')},
 'AerSale': {('https://twitter.com/aersale', 'aersale')},
 'ManpowerGroup': {('https://twitter.com/ManpowerGroup', 'ManpowerGroup')},
 'Penske': {('h

Now it's time to add a new row for each subsidiary's Twitter account to our dataframe! Let's make a copy of our dataframe.


In [10]:
import warnings
warnings.filterwarnings('ignore') # ignore warnings about append

df_mergr_subsidiaries = df_mergr_manual.copy(deep=True)

for name in name_to_twitters_map:
    twitter_tuples = name_to_twitters_map[name]
    num_twitters_scraped = len(twitter_tuples)
    
    # If a single Twitter account was scraped, that's probably just the parent company's Twitter account,
    # which is already in our dataframe, so we don't want to add a new row for that.
    if num_twitters_scraped > 1:
        # Get index of the parent company's row by finding the first row that matches the company Name
        parent_company_row_index = df_mergr_manual.loc[df_mergr_manual["Name"] == name].index[0]
        
        # Copy that old parent company row to a new subsidiary row
        parent_company_row = df_mergr_manual.loc[parent_company_row_index]
        subsidiary_row = parent_company_row.copy()
        
        # Iterate thru the (Twitter URL, Twitter Handle) tuples for this parent company
        for twitter_url, twitter_handle in twitter_tuples:
            if twitter_url != parent_company_row["Twitter URL"] and twitter_handle.lower() != parent_company_row["Twitter Handle"].lower():
                # It's a subsidiary Twitter URL, not the parent Twitter URL!
                # Update the Twitter URL, Twitter Handle, and Subsidiary columns for the new row
                subsidiary_row["Twitter URL"] = twitter_url
                subsidiary_row["Twitter Handle"] = twitter_handle
                subsidiary_row["Subsidiary"] = True
                        
                # Add subsidiary as new row in df
                df_mergr_subsidiaries = df_mergr_subsidiaries.append(subsidiary_row)

In [11]:
# Sort dataframe alphabetically, primarily by company Security, and secondarily by Subsidiary column value
# The lambda function ensures that the sort is case-insensitive
# So that parent companies and their subsidiaries are in consecutive rows
df_mergr_subsidiaries["Subsidiary"] = df_mergr_subsidiaries["Subsidiary"].map({True: 'True', False: 'False'}) # convert Subsidiary column to strings to do the sorting
df_mergr_subsidiaries = df_mergr_subsidiaries.sort_values(["Name", "Subsidiary"], key=lambda col: col.str.lower())
df_mergr_subsidiaries["Subsidiary"] = df_mergr_subsidiaries["Subsidiary"].map({'True': True, 'False': False}) # convert Subsidiary column back to bools

# Save as CSV
df_mergr_subsidiaries.to_csv("mergr_twitter_subsidiaries.csv")

In [12]:
# Subtract 1 because 1 row is the column headings
print("Number of rows in original df: ", len(df_mergr_manual) - 1)
print("Number of rows in new df: ", len(df_mergr_subsidiaries) - 1)

Number of rows in original df:  52
Number of rows in new df:  59


Looks like we found 7 new Twitter accounts! Does this mean 7 potential subsidiaries?



We may want to manually check to ensure that these aren't career-specific Twitter accounts or unrelated Twitter accounts



In [13]:
df_mergr_subsidiaries_manual = pd.read_csv('mergr_twitter_subsidiaries_manual.csv')
print("Number of rows in manually corrected df: ", len(df_mergr_subsidiaries_manual) - 1)

Number of rows in manually corrected df:  53


From our manual corrections, it looks like we actually found just 1 new subsidiary with a Twitter account.


In [14]:
# Drop duplicates (rows that have the same Twitter Handle values) except for the first occurrence,
# and don't drop any rows that have a null value for the Twitter Handle column.
df_mergr_subsidiaries_manual_no_dups = df_mergr_subsidiaries_manual[(~df_mergr_subsidiaries_manual['Twitter Handle'].duplicated()) | df_mergr_subsidiaries_manual['Twitter Handle'].isna()]


In [15]:
# Subtract 1 because 1 row is the column headings
print("Number of rows in original df: ", len(df_mergr_subsidiaries_manual) - 1)
print("Number of rows in new df: ", len(df_mergr_subsidiaries_manual_no_dups) - 1)

Number of rows in original df:  53
Number of rows in new df:  53


It looks like we didn't have any rows with duplicate Twitter Handles to begin with!